<a href="https://colab.research.google.com/github/gorzanskik-ai/short-term-stocks-price-prediction/blob/main/investment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import numpy as np
import plotly.express as px

np.random.seed(42)

import matplotlib.pyplot as plt
import plotly.graph_objects as go

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_squared_error

Mounted at /content/drive


In [ ]:
from tensorflow.keras.models import load_model

classifier = load_model("/content/drive/MyDrive/master's thesis/day/classification.h5")
regressor = load_model("/content/drive/MyDrive/master's thesis/day/regression.h5")

In [ ]:
path1 = "/content/drive/MyDrive/master's thesis/day/investment_reg.csv"
df_raw = pd.read_csv(path1)
df_raw.index = df_raw.Date
df_raw = df_raw.drop('Date', axis=1)
df_reg = df_raw.copy()
df_reg = df_reg['2022-05-02':]

path2 = "/content/drive/MyDrive/master's thesis/day/investment_cls.csv"
df_raw = pd.read_csv(path2)
df_raw.index = df_raw.Date
df_raw = df_raw.drop('Date', axis=1)
df_cls = df_raw.copy()
df_cls = df_cls['2022-05-02':]

Classification

In [ ]:
X_cls = df_cls.drop('TARGET', axis=1).copy()
y_cls = df_cls['TARGET'].copy()

features = ['Day', 'Month', 'Year', 'RSI', 'MFI', 'CMO', 'UO', 'ADX']
scaler = MinMaxScaler()
scaler.fit(X_cls[features])
X_cls[features] = scaler.transform(X_cls[features])


X_cls = X_cls.to_numpy()
y_true_cls = y_cls.to_numpy()

X_cls = X_cls.reshape(X_cls.shape[0], 1, X_cls.shape[1])

In [ ]:
y_pred_cls = classifier.predict(X_cls)
y_pred_cls = y_pred_cls.reshape(y_pred_cls.shape[0])

pred = pd.DataFrame()
pred['0'] = np.absolute(y_pred_cls - 1)
pred['1'] = y_pred_cls
pred = pred.to_numpy()
y_pred_cls = np.argmax(pred, axis=1)
print(f'Accuracy score: {accuracy_score(y_true_cls, y_pred_cls)}')

Accuracy score: 0.75


Regression

In [ ]:
X = df_reg.drop('TARGET', axis=1).copy()
y = df_reg['TARGET'].copy()

features = ['Day', 'Month', 'Year', 'RSI', 'MFI', 'CMO', 'UO', 'ADX'] 
scaler = MinMaxScaler()
scaler.fit(X[features])
X[features] = scaler.transform(X[features])


X = X.to_numpy()
y_true = y.to_numpy()

X = X.reshape(X.shape[0], 1, X.shape[1])

In [ ]:
y_pred = regressor.predict(X).flatten()

y_pred = y_pred.reshape(y_pred.shape[0])

print('mape', mean_absolute_percentage_error(y_true, y_pred))
print('rmse', mean_squared_error(y_true, y_pred, squared=False))

mape 0.0163968953062983
rmse 80.13785660673425


In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x = np.arange(0, y_true.shape[0]), y = y_true, mode='lines', name='y_true'))
fig.add_trace(go.Scatter(x = np.arange(0, y_true.shape[0]), y = y_pred, mode='lines', name='y_pred'))
fig.update_layout(width=1000, height=500, title='', xaxis_title='Time', yaxis_title='Close')
fig.show()

Badania

In [ ]:
df_all = df_reg[['Open', 'Low', 'High', 'Close', 'TARGET']]
df_all['Reg'] = y_pred
df_all['Cls'] = y_pred_cls

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
df_all.head()

,Open,Low,High,Close,TARGET,Reg,Cls
Date,,,,,,,
2022-05-02,4130.61,4062.51,4169.81,4155.38,4175.48,4138.665039,1
2022-05-03,4159.78,4147.08,4200.10,4175.48,4300.17,4182.147949,1
2022-05-04,4181.18,4148.91,4307.66,4300.17,4146.87,4258.976562,1
2022-05-05,4270.43,4106.01,4270.43,4146.87,4123.34,4187.181641,0
2022-05-06,4128.17,4067.91,4157.69,4123.34,3991.24,4125.707520,0


Regresja

In [ ]:
money = 10000
lewar = 10
stocks = 0

for index, row in df_all.iterrows():

    if row['Reg'] < row['Close']:
        rate = row['TARGET'] / row['Close']
        if rate > 1:
            money = money + (money * (rate - 1)) * lewar
        else:
            money = money - (money * (1 - rate)) * lewar
    else:
        rate = row['TARGET'] / row['Close']

        if rate < 1:
            money = money + (money * (1 - rate)) * lewar
        else:
            money = money - (money * (rate - 1)) * lewar

money

6960.9048457262925

Klasyfikacja

In [ ]:
money = 10000
lewar = 2
stocks = 0

for index, row in df_all.iterrows():

    if row['Cls'] == 0:
        rate = row['TARGET'] / row['Close']
        if rate > 1:
            money = money + (money * (rate - 1)) * lewar
        else:
            money = money - (money * (1 - rate)) * lewar
    else:
        rate = row['TARGET'] / row['Close']

        if rate < 1:
            money = money + (money * (1 - rate)) * lewar
        else:
            money = money - (money * (rate - 1)) * lewar

money

9383.393391545305

all

In [ ]:
money = 10000
lewar = 10
stocks = 0

for index, row in df_all.iterrows():

    if row['Cls'] == 1 and row['Reg'] < row['Close']:
        rate = row['TARGET'] / row['Close']
        if rate > 1:
            money = money + (money * (rate - 1)) * lewar
        else:
            money = money - (money * (1 - rate)) * lewar
    elif row['Cls'] == 0 and row['Reg'] > row['Close']:
        rate = row['TARGET'] / row['Close']

        if rate < 1:
            money = money + (money * (1 - rate)) * lewar
        else:
            money = money - (money * (rate - 1)) * lewar

money

7659.689452880328

In [ ]:
df_all[(df_all['Cls'] == 1) & (df_all['Reg'] > df_all['Close'])]

,Open,Low,High,Close,TARGET,Reg,Cls
Date,,,,,,,
2022-05-03,4159.78,4147.08,4200.10,4175.48,4300.17,4182.147949,1
2022-05-19,3899.00,3876.58,3945.96,3900.79,3901.36,3915.276855,1


In [ ]:
df_all[(df_all['Cls'] == 0) & (df_all['Reg'] < df_all['Close'])]

,Open,Low,High,Close,TARGET,Reg,Cls
Date,,,,,,,
2022-05-20,3927.76,3810.32,3943.42,3901.36,3973.75,3895.713867,0
2022-05-24,3942.94,3875.13,3955.68,3941.48,3978.73,3933.566406,0


In [ ]:
money = 10000
lewar = 1
stocks = 0

for index, row in df_all.iterrows():

    if row['Reg'] > row['Close']:
        stocks = money / row['Close']
        money = stocks * row['TARGET']
    else:
        rate = row['TARGET'] / row['Close']

        if rate < 1:
            money = money + (money * (1 - rate))
        else:
            money = money - (money * (rate - 1))

money

9774.790974195932